# Actors


In this tutorial, we demonstrate how getML can be applied in an e-commerce context. Using a dataset of about 400,000 orders, our goal is to predict whether an order will be cancelled.

We also show that we can significantly improve our results by using getML's built-in hyperparameter tuning routines.

Summary:

- Prediction type: __Classification model__
- Domain: __E-commerce__
- Prediction target: __The gender of an actor__ 
- Population size: __817718__

_Author: Dr. Patrick Urbanke_

# Background

The data set contains about 400,000 orders from a British online retailer. Each order consists of a product that has been ordered and a corresponding quantity. Several orders can be summarized onto a single invoice. The goal is to predict whether an order will be cancelled.

Because the company mainly sells to other businesses, the cancellation rate is relatively low, namely 1.83%.

The data set has been originally collected for this study:

Daqing Chen, Sai Liang Sain, and Kun Guo, Data mining for the online retail industry: A case study of RFM model-based customer segmentation using data mining, Journal of Database Marketing and Customer Strategy Management, Vol. 19, No. 3, pp. 197-208, 2012 (Published online before print: 27 August 2012. doi: 10.1057/dbm.2012.17).

It has been downloaded from the UCI Machine Learning Repository:

Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science.

### A web frontend for getML

The getML monitor is a frontend built to support your work with getML. The getML monitor displays information such as the imported data frames, trained pipelines and allows easy data and feature exploration. You can launch the getML monitor [here](http://localhost:1709).

### Where is this running?

Your getML live session is running inside a docker container on [mybinder.org](https://mybinder.org/), a service built by the Jupyter community and funded by Google Cloud, OVH, GESIS Notebooks and the Turing Institute. As it is a free service, this session will shut down after 10 minutes of inactivity.

# Analysis

Let's get started with the analysis and set up your session:

In [2]:
import copy
import os
from urllib import request

import numpy as np
import pandas as pd
from IPython.display import Image
import matplotlib.pyplot as plt
plt.style.use('seaborn')
%matplotlib inline  

from sklearn.feature_extraction.text import CountVectorizer

import getml

getml.engine.set_project('actors')


Connected to project 'actors'


Tuning is effective at improving our results, but it takes quite long, so we want to make it optional:

## 1. Loading data

### 1.1 Download from source

We begin by downloading the data from the source file:

In [ ]:
conn = getml.database.connect_mariadb(
    host="relational.fit.cvut.cz",
    dbname="imdb_ijs",
    port=3306,
    user="guest",
    password="relational"
)

conn

In [ ]:
def load_if_needed(name):
    """
    Loads the data from the relational learning
    repository, if the data frame has not already
    been loaded.
    """
    if not getml.data.exists(name):
        data_frame = getml.data.DataFrame.from_db(
            name=name,
            table_name=name,
            conn=conn
        )
        data_frame.save()
    else:
        data_frame = getml.data.load_data_frame(name)
    return data_frame

In [ ]:
actors = load_if_needed("actors")
roles = load_if_needed("roles")
movies = load_if_needed("movies")
movies_genres = load_if_needed("movies_genres")

In [ ]:
actors

In [ ]:
roles

In [ ]:
movies

In [ ]:
movies_genres

### 1.2 Prepare data for getML

In [ ]:
def fit_transform_count_vectorizer(text_fields):
    count_vectorizer = CountVectorizer(min_df=11)
    transformed = count_vectorizer.fit_transform(text_fields)
    return count_vectorizer.inverse_transform(transformed)

In [ ]:
def make_data_frame(transformed_text_fields, df_name):
    join_keys = []
    indices = []
    words = []
    #
    for i, trans in enumerate(transformed_text_fields):
        join_keys += [i] * len(trans)
        indices += range(len(trans.tolist()))
        words += trans.tolist()
    #
    data_frame = pd.DataFrame()
    data_frame["join_key"] = join_keys
    data_frame["index"] = indices
    data_frame["words"] = words
    #
    return getml.data.DataFrame.from_pandas(data_frame, df_name)

In [ ]:
def process_text_fields(text_fields, df_name):
    transformed_text_fields = fit_transform_count_vectorizer(text_fields)
    return make_data_frame(transformed_text_fields, df_name)

In [ ]:
first_names = actors.first_name.to_numpy()
first_names = process_text_fields(first_names, "first_names")
first_names

In [ ]:
role = roles.role.to_numpy()
role = process_text_fields(role, "role")
role

getML requires that we define *roles* for each of the columns.

In [ ]:
actors["target"] = (actors.gender == 'F').as_num()
actors["rownum"] = actors.rowid()

In [ ]:
actors.set_role(["id", "rownum"], getml.data.roles.join_key)
actors.set_role("target", getml.data.roles.target)

In [ ]:
roles["rownum"] = roles.rowid()
roles.set_role(["actor_id", "movie_id", "rownum"], getml.data.roles.join_key)

In [ ]:
movies.set_role("id", getml.data.roles.join_key)
movies.set_role(["year", "rank"], getml.data.roles.numerical)

In [ ]:
movies_genres.set_role("movie_id", getml.data.roles.join_key)
movies_genres.set_role("genre", getml.data.roles.categorical)

In [ ]:
first_names.set_role("join_key", getml.data.roles.join_key)
first_names.set_role("index", getml.data.roles.numerical)
first_names.set_role("words", getml.data.roles.categorical)

In [ ]:
role.set_role("join_key", getml.data.roles.join_key)
role.set_role("index", getml.data.roles.numerical)
role.set_role("words", getml.data.roles.categorical)

The *StockCode* is a 5-digit code that uniquely defines a product. It is hierarchical, meaning that every digit has a meaning. We want to make use of that, so we assign a unit to the stock code, which we can reference in our preprocessors.

Let's take a look at what we have done so far:

In [ ]:
random = actors.random()

is_training = (random < 0.7)
is_validation = (~is_training & (random < 0.85))
is_test = (~is_training & ~is_validation)

data_train = actors.where("data_train", is_training)
data_validation = actors.where("data_validation", is_validation)
data_test = actors.where("data_test", is_test)

## 2. Predictive modelling

We loaded the data and defined the roles and units. Next, we create a getML pipeline for relational learning.

### 2.1 Define relational model

To get started with relational learning, we need to specify the data model.

In our case, there are two joins we are interested in: 

1) We want to take a look at all of the other orders on the same invoice.

2) We want to check out how often a certain customer has cancelled orders in the past. Here, we limit ourselves to the last 90 days. To avoid data leaks, we set a horizon of one day.

In [3]:
actors_ph = getml.data.Placeholder('actors')
first_names_ph = getml.data.Placeholder('first_names')
roles_ph = getml.data.Placeholder('roles')
role_ph = getml.data.Placeholder('role')
movies_ph = getml.data.Placeholder('movies')
movie_genres_ph = getml.data.Placeholder('movie_genres')

#actors_ph.join(
#    first_names_ph,
#    join_key='rownum',
#    other_join_key='join_key'
#)

actors_ph.join(
    roles_ph,
    join_key='id',
    other_join_key='actor_id'
)

roles_ph.join(
    role_ph,
    join_key='rownum',
    other_join_key='join_key'
)

roles_ph.join(
    movies_ph,
    join_key='movie_id',
    other_join_key='id',
    relationship=getml.data.relationship.many_to_one
)

movies_ph.join(
    movie_genres_ph,
    join_key='id',
    other_join_key='movie_id'
)

actors_ph

placeholder   other placeholder   allow lagged targets   horizon   join keys used   ...   
actors        roles               False                  0.0       id               ...   
roles         role                False                  0.0       rownum           ...   
roles         movies              False                  0.0       movie_id         ...   
movies        movie_genres        False                  0.0       id               ...   



other join keys used   other time stamps used   relationship   time stamps used   
actor_id                                        many-to-many                      
join_key                                        many-to-many                      
id                                              many-to-one                       
movie_id                                        many-to-many                      



other join keys used   upper time stamps used   
actor_id                                        
join_key                                        
id                                              
movie_id

In [4]:
getml.data.diagram._DataModel(actors_ph).to_html()

'<div style="height:210px;width:1660px;position:relative;"><svg height="200" width="1650"><rect y="0" x="500" rx="10" ry="10" width="150" height="90" style="fill:#6829c2;stroke-width:0;" /><text y="73.8"" x="575.0" dominant-baseline="middle" text-anchor="middle" fill="white">role</text><rect x="551" y="10" rx="4" ry="4" width="48" height="48" style=" fill:#6829c2;stroke:#ffffff;stroke-width:3;" /><line x1="567.0" y1="10" x2="567.0" y2="58" style="stroke:white;stroke-width:3" /><line x1="583.0" y1="10" x2="583.0" y2="58" style="stroke:white;stroke-width:3" /><line x1="551" y1="26.0" x2="599" y2="26.0" style="stroke:white;stroke-width:3" /><line x1="551" y1="42.0" x2="599" y2="42.0" style="stroke:white;stroke-width:3" /><rect y="110" x="0" rx="10" ry="10" width="150" height="90" style="fill:#6829c2;stroke-width:0;" /><text y="183.8"" x="75.0" dominant-baseline="middle" text-anchor="middle" fill="white">movie_genres</text><rect x="51" y="120" rx="4" ry="4" width="48" height="48" style=" f

In [5]:
from IPython.core.display import HTML

HTML('<div style="height:210px;width:1660px;position:relative;"><svg height="200" width="1650"><rect y="0" x="500" rx="10" ry="10" width="150" height="90" style="fill:#6829c2;stroke-width:0;" /><text y="73.8"" x="575.0" dominant-baseline="middle" text-anchor="middle" fill="white">role</text><rect x="551" y="10" rx="4" ry="4" width="48" height="48" style=" fill:#6829c2;stroke:#ffffff;stroke-width:3;" /><line x1="567.0" y1="10" x2="567.0" y2="58" style="stroke:white;stroke-width:3" /><line x1="583.0" y1="10" x2="583.0" y2="58" style="stroke:white;stroke-width:3" /><line x1="551" y1="26.0" x2="599" y2="26.0" style="stroke:white;stroke-width:3" /><line x1="551" y1="42.0" x2="599" y2="42.0" style="stroke:white;stroke-width:3" /><rect y="110" x="0" rx="10" ry="10" width="150" height="90" style="fill:#6829c2;stroke-width:0;" /><text y="183.8"" x="75.0" dominant-baseline="middle" text-anchor="middle" fill="white">movie_genres</text><rect x="51" y="120" rx="4" ry="4" width="48" height="48" style=" fill:#6829c2;stroke:#ffffff;stroke-width:3;" /><line x1="67.0" y1="120" x2="67.0" y2="168" style="stroke:white;stroke-width:3" /><line x1="83.0" y1="120" x2="83.0" y2="168" style="stroke:white;stroke-width:3" /><line x1="51" y1="136.0" x2="99" y2="136.0" style="stroke:white;stroke-width:3" /><line x1="51" y1="152.0" x2="99" y2="152.0" style="stroke:white;stroke-width:3" /><rect y="110" x="500" rx="10" ry="10" width="150" height="90" style="fill:#6829c2;stroke-width:0;" /><text y="183.8"" x="575.0" dominant-baseline="middle" text-anchor="middle" fill="white">movies</text><rect x="551" y="120" rx="4" ry="4" width="48" height="48" style=" fill:#6829c2;stroke:#ffffff;stroke-width:3;" /><line x1="567.0" y1="120" x2="567.0" y2="168" style="stroke:white;stroke-width:3" /><line x1="583.0" y1="120" x2="583.0" y2="168" style="stroke:white;stroke-width:3" /><line x1="551" y1="136.0" x2="599" y2="136.0" style="stroke:white;stroke-width:3" /><line x1="551" y1="152.0" x2="599" y2="152.0" style="stroke:white;stroke-width:3" /><rect y="110" x="1000" rx="10" ry="10" width="150" height="90" style="fill:#6829c2;stroke-width:0;" /><text y="183.8"" x="1075.0" dominant-baseline="middle" text-anchor="middle" fill="white">roles</text><rect x="1051" y="120" rx="4" ry="4" width="48" height="48" style=" fill:#6829c2;stroke:#ffffff;stroke-width:3;" /><line x1="1067.0" y1="120" x2="1067.0" y2="168" style="stroke:white;stroke-width:3" /><line x1="1083.0" y1="120" x2="1083.0" y2="168" style="stroke:white;stroke-width:3" /><line x1="1051" y1="136.0" x2="1099" y2="136.0" style="stroke:white;stroke-width:3" /><line x1="1051" y1="152.0" x2="1099" y2="152.0" style="stroke:white;stroke-width:3" /><rect y="110" x="1500" rx="10" ry="10" width="150" height="90" style="fill:#6829c2;stroke-width:0;" /><text y="183.8"" x="1575.0" dominant-baseline="middle" text-anchor="middle" fill="white">actors</text><rect x="1551" y="120" rx="4" ry="4" width="48" height="48" style=" fill:#6829c2;stroke:#ffffff;stroke-width:3;" /><line x1="1567.0" y1="120" x2="1567.0" y2="168" style="stroke:white;stroke-width:3" /><line x1="1583.0" y1="120" x2="1583.0" y2="168" style="stroke:white;stroke-width:3" /><line x1="1551" y1="136.0" x2="1599" y2="136.0" style="stroke:white;stroke-width:3" /><line x1="1551" y1="152.0" x2="1599" y2="152.0" style="stroke:white;stroke-width:3" /><line x1="150" y1="153.0" x2="490" y2="153.0" style="stroke:#808080;;stroke-width:4" /><polygon points="500, 153.0 490, 147.0 490, 159.0 " style="fill:#808080;;stroke-width:0;" /><rect y="120.0" x="249.0" rx="10" ry="10" width="150" height="70" style="fill:#6829c2;stroke-width:0;" /><text dominant-baseline="middle" text-anchor="middle" fill="white"><tspan y="155.0"" x="324.0" font-size="7pt" >movie_id = id</tspan></text><line x1="650" y1="43.0" x2="1073.0" y2="43.0" style="stroke:#808080;;stroke-width:4" /><line x1="1073.0" y1="41.0" x2="1073.0" y2="100" style="stroke:#808080;;stroke-width:4" /><polygon points="1073.0, 110 1067.0, 100 1079.0, 100 " style="fill:#808080;;stroke-width:0;" /><rect y="10.0" x="749.0" rx="10" ry="10" width="150" height="70" style="fill:#6829c2;stroke-width:0;" /><text dominant-baseline="middle" text-anchor="middle" fill="white"><tspan y="45.0"" x="824.0" font-size="7pt" >join_key = rownum</tspan></text><line x1="650" y1="153.0" x2="990" y2="153.0" style="stroke:#808080;;stroke-width:4" /><polygon points="1000, 153.0 990, 147.0 990, 159.0 " style="fill:#808080;;stroke-width:0;" /><rect y="120.0" x="749.0" rx="10" ry="10" width="150" height="70" style="fill:#6829c2;stroke-width:0;" /><text dominant-baseline="middle" text-anchor="middle" fill="white"><tspan y="150.0"" x="824.0" font-size="7pt" >id = movie_id</tspan><tspan y="160.0"" x="824.0" font-size="7pt" >Relationship: many-to-one</tspan></text><line x1="1150" y1="153.0" x2="1490" y2="153.0" style="stroke:#808080;;stroke-width:4" /><polygon points="1500, 153.0 1490, 147.0 1490, 159.0 " style="fill:#808080;;stroke-width:0;" /><rect y="120.0" x="1249.0" rx="10" ry="10" width="150" height="70" style="fill:#6829c2;stroke-width:0;" /><text dominant-baseline="middle" text-anchor="middle" fill="white"><tspan y="155.0"" x="1324.0" font-size="7pt" >actor_id = id</tspan></text></svg></div>')

### 2.2 getML pipeline

<!-- #### 2.1.1  -->
__Set-up the feature learner & predictor__

We have mentioned that the *StockCode* is a hierarchical code. To make use of that fact, we use getML's substring preprocessor, extracting the first digit, the first two digits etc. Since we have assigned the unit *code* to the *StockCode*, the preprocessors know which column they should be applied to.

In [ ]:
relboost = getml.feature_learning.RelboostModel(
    num_features=30,
    num_subfeatures=10,
    loss_function=getml.feature_learning.loss_functions.CrossEntropyLoss,
    seed=4367,
    num_threads=1,
    sampling_factor=10,
    max_depth=6
)

predictor = getml.predictors.XGBoostClassifier()

__Build the pipeline__

In [ ]:
pipe = getml.pipeline.Pipeline(
    tags=['relboost'],
    population=actors_ph,
    peripheral=[roles_ph, movies_ph, movie_genres_ph, first_names_ph, role_ph],
    feature_learners=[relboost],
    predictors=[predictor]
)

### 2.3 Model training

In [ ]:
pipe.check(data_train, {"roles": roles, "movies": movies, "movie_genres": movies_genres, "first_names": first_names, "role": role})

In [ ]:
pipe.fit(data_train, {"roles": roles, "movies": movies, "movie_genres": movies_genres, "first_names": first_names, "role": role})

### 2.4 Model evaluation

In [ ]:
in_sample = pipe.score(data_train, {"roles": roles, "movies": movies, "movie_genres": movies_genres, "first_names": first_names, "role": role})

out_of_sample = pipe.score(data_test, {"roles": roles, "movies": movies, "movie_genres": movies_genres, "first_names": first_names, "role": role})

pipe.scores

### 2.6 Studying features

__Feature correlations__

We want to analyze how the features are correlated with the target variable.

In [ ]:
names, correlations = pipe.features.correlations()

plt.subplots(figsize=(20, 10))

plt.bar(names, correlations, color='#6829c2')

plt.title('Feature Correlations')
plt.xlabel('Features')
plt.ylabel('Correlations')
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
pipe.features.to_sql().save("features.sql")

__Feature importances__
 
Feature importances are calculated by analyzing the improvement in predictive accuracy on each node of the trees in the XGBoost predictor. They are then normalized, so that all importances add up to 100%.

In [ ]:
names, importances = pipe.features.importances()

plt.subplots(figsize=(20, 10))

plt.bar(names, importances, color='#6829c2')

plt.title('Feature Importances')
plt.xlabel('Features')
plt.ylabel('Importances')
plt.xticks(rotation='vertical')
plt.show()

most_important = names[0]

__Column importances__

Because getML uses relational learning, we can apply the principles we used to calculate the feature importances to individual columns as well.

As we can see, the *StockCode* contributes about 50% of the predictive accuracy.

In [ ]:
names, importances = pipe.columns.importances()

plt.subplots(figsize=(20, 10))

plt.bar(names, importances, color='#6829c2')

plt.title('Columns importances')
plt.xlabel('Columns')
plt.ylabel('Importances')
plt.xticks(rotation='vertical')
plt.show()

most_important = names[0]

__Transpiling the learned features__

We can also transpile the learned features to SQLite3 code. We want to show the two most important features. That is why we call the `.features.importances().` method again. The names that are returned are already sorted by importance.

In [ ]:
names, _ = pipe.features.importances()

pipe.features.to_sql()[names[0]]

In [ ]:
names, _ = pipe.features.importances()

pipe.features.to_sql()[names[1]]

## 3. Conclusion

In this notebook we have demonstrated how getML can be applied to an e-commerce setting. In particular, we have seen how results can be improved using the built-in hyperparamater tuning routines.

# Next Steps

This tutorial went through the basics of applying getML to relational data. If you want to learn more about getML, here are some additional tutorials and articles that will help you:

__Tutorials:__
* [Loan default prediction: Introduction to relational learning](loans_demo.ipynb)
* [Occupancy detection: A multivariate time series example](occupancy_demo.ipynb)  
* [Expenditure categorization: Why relational learning matters](consumer_expenditures_demo.ipynb)
* [Disease lethality prediction: Feature engineering and the curse of dimensionality](atherosclerosis_demo.ipynb)
* [Traffic volume prediction: Feature engineering on multivariate time series](interstate94_demo.ipynb)
* [Air pollution prediction: Why feature learning outperforms brute-force approaches](air_pollution_demo.ipynb) 


__User Guides__ (from our [documentation](https://docs.getml.com/latest/)):
* [Feature learning with Multirel](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html#multirel)
* [Feature learning with Relboost](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html#relboost)


# Get in contact

If you have any question schedule a [call with Alex](https://go.getml.com/meetings/alexander-uhlig/getml-demo), the co-founder of getML, or write us an [email](team@getml.com). Prefer a private demo of getML? Just contact us to make an appointment.